In [ ]:
!pip install tensorflow-text

In [ ]:
import tensorflow as tf
import tensorflow_hub as hub
import tensorflow_text
from scipy import spatial
import numpy as np


In [ ]:
module_url_1 = "https://tfhub.dev/google/universal-sentence-encoder-multilingual/3"
module_url_2 = "https://tfhub.dev/google/universal-sentence-encoder-multilingual-large/3"
model_1 = hub.load(module_url_1)
model_2 = hub.load(module_url_2)

In [ ]:
input = "agreement"

embedding_input_1 = model_1(input)
display(embedding_input_1.shape)
embedding_input_2 = model_2(input)
display(embedding_input_2.shape)

TensorShape([1, 512])

TensorShape([1, 512])

In [ ]:
input2 = "Einigung"
embedding_input_2_1 = model_1(input2)
display(embedding_input_2_1.shape)
embedding_input_2_3 = model_2(input2)
display(embedding_input_2_3.shape)

TensorShape([1, 512])

TensorShape([1, 512])

In [ ]:
 
 result = 1 - spatial.distance.cosine(embedding_input_1, embedding_input_2_1)
 display(result)
 
 result = 1 - spatial.distance.cosine(embedding_input_2, embedding_input_2_3)
 display(result)

0.6080600619316101

0.5760625600814819

In [ ]:
text_input = tf.keras.layers.Input(shape=(), dtype=tf.string)
preprocessor = hub.KerasLayer(
    "https://tfhub.dev/tensorflow/bert_multi_cased_preprocess/3")
encoder_inputs = preprocessor(text_input)
encoder = hub.KerasLayer(
    "https://tfhub.dev/tensorflow/bert_multi_cased_L-12_H-768_A-12/4",
    trainable=True)
outputs = encoder(encoder_inputs)
pooled_output = outputs["pooled_output"]      # [batch_size, 768].
sequence_output = outputs["sequence_output"]  # [batch_size, seq_length, 768].


In [ ]:
embedding_model = tf.keras.Model(text_input, pooled_output)
emb1 = tf.constant(["car"])
emb2 = tf.constant(["König"])

import numpy as np
emb1 = np.array(embedding_model(emb1))
emb2 = np.array(embedding_model(emb2))
print(emb1.shape)
print(emb2.shape)


(1, 768)
(1, 768)


In [ ]:
result = 1 - spatial.distance.cosine(emb1, emb2)
display(result)

0.8962827920913696

###### load NRC intensity lexicon

In [ ]:
%cd 

/root


In [ ]:
%cd /content/drive/MyDrive/emotion_flow_thesis/nrclexicon

/content/drive/MyDrive/emotion_flow_thesis/nrclexicon


In [ ]:
import pandas as pd
filepath = "data/NRC-Emotion-Intensity-Lexicon-v1-ForVariousLanguages.txt"
emolex_df = pd.read_csv(filepath, skiprows=0, sep='\t', keep_default_na=False)
emolex_df_short = emolex_df[["word", "emotion", "emotion-intensity-score"]]


###### load NRC lexicon (not intensity)

In [ ]:
%cd
%cd /content/drive/MyDrive/emotion_flow_thesis/nrclexicon/
import pandas as pd
filepath = "data/NRC-Emotion-Lexicon-v0.92-In105Languages-Nov2017Translations.xlsx"
emolex_df = pd.read_excel(filepath)  
emolex_df

In [ ]:
emolex_df.columns
emolex_df_short = emolex_df[['English (en)', 'German (de)', 'Positive', 	'Negative', 	'Anger', 	'Anticipation', 	'Disgust', 	'Fear', 	'Joy', 	'Sadness', 	'Surprise', 	'Trust']]
emolex_df_short

###### Create NRC lexicon with embeddings

In [ ]:
embeddings_df = pd.DataFrame(columns=range(512))
display(embeddings_df)
count = 0
for index, row in emolex_df_short.iterrows():
    word = row["English (en)"]

    input = tf.constant([word])

    embedding = np.array(embedding_model(input))
    
    embedding = list(embedding[0])

    embeddings_df.loc[len(embeddings_df.index)] = embedding
    print(count)
    count += 1
    if count == 5:
      break
    print(word)
display(embeddings_df)
    

In [ ]:
embeddings_df.to_csv("embeddings.csv", index=False)
embeddings_df.to_csv("embeddings_index.csv", index=True)

In [ ]:
emolex_df_short

,English (en),German (de),Positive,Negative,Anger,Anticipation,Disgust,Fear,Joy,Sadness,Surprise,Trust
0,aback,zurück,0,0,0,0,0,0,0,0,0,0
1,abacus,Abakus,0,0,0,0,0,0,0,0,0,1
2,abandon,verlassen,0,1,0,0,0,1,0,1,0,0
3,abandoned,verlassen,0,1,1,0,0,1,0,1,0,0
4,abandonment,Aufgabe,0,1,1,0,0,1,0,1,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...
14177,zone,Zone,0,0,0,0,0,0,0,0,0,0
14178,zoo,Zoo,0,0,0,0,0,0,0,0,0,0
14179,zoological,zoologisch,0,0,0,0,0,0,0,0,0,0
14180,zoology,Zoologie,0,0,0,0,0,0,0,0,0,0


In [ ]:
for i in range(5):
  display(str((emolex_df_short.iloc[8480+i])['English (en)']))

'nudity'

'nugget'

'nuisance'

'nul'

'nan'

In [ ]:

embeddings_df = pd.DataFrame(columns=range(512))
display(embeddings_df)
count = 0
for index, row in emolex_df_short.iterrows():
    word = row["English (en)"]
    
    embedding = np.array(model_1(str(word)))    
    embedding = list(embedding[0])
    embeddings_df.loc[len(embeddings_df.index)] = embedding
    print(count)
    count += 1
    
display(embeddings_df)

In [ ]:
display(embeddings_df)

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,...,472,473,474,475,476,477,478,479,480,481,482,483,484,485,486,487,488,489,490,491,492,493,494,495,496,497,498,499,500,501,502,503,504,505,506,507,508,509,510,511
0,0.149352,-0.003459,-0.065422,-0.028715,-0.051330,0.006737,0.058559,0.095040,-0.021063,-0.044034,-0.041016,-0.001103,-0.041896,-0.025348,-0.036527,-0.017931,-0.023806,0.048461,-0.059002,0.087159,-0.030405,0.141753,0.056645,-0.004798,0.064418,0.020047,0.013349,0.030566,0.005457,0.016997,0.038505,0.070150,0.001943,-0.061528,-0.018682,-0.062276,-0.058789,0.038703,0.018262,0.026396,...,0.007557,-0.019122,0.075607,0.017159,0.013709,-0.038075,0.044675,0.047889,-0.031919,0.144506,0.023908,-0.022499,-0.018366,0.001174,0.023365,-0.117180,0.004208,0.032454,0.005253,0.015592,0.005585,-0.000541,-0.017391,-0.038685,-0.060557,0.013524,-0.008002,0.042231,-0.057411,-0.048669,-0.086644,-0.014687,0.062922,-0.002512,-0.024360,-0.025476,0.013133,0.086978,0.012901,0.062648
1,0.107717,-0.018898,-0.011394,0.050276,-0.048540,-0.012108,-0.056628,-0.039981,-0.034419,-0.030321,-0.026917,-0.047560,-0.075619,0.012728,-0.092619,0.009156,0.052593,0.016761,0.006242,0.060279,0.045265,0.100351,0.074628,-0.077011,-0.004467,-0.029406,-0.022527,0.040639,0.017914,-0.030976,-0.009833,0.031153,0.063423,-0.075497,0.039019,-0.066110,-0.044073,-0.022280,-0.048912,0.043776,...,-0.055137,-0.027894,0.012977,0.021505,0.007632,-0.005122,0.019104,-0.032691,-0.003106,0.090437,0.071537,0.016522,0.051937,0.013511,0.063138,-0.090772,-0.029901,0.080202,0.016331,-0.036264,-0.006395,-0.064492,0.048998,0.013739,-0.040107,-0.021306,0.050751,-0.003735,-0.041972,-0.000626,-0.089578,0.018679,0.061766,-0.019972,-0.077222,0.031832,-0.029841,0.081567,0.045708,0.065206
2,0.107151,0.016580,-0.046364,-0.013345,-0.059200,0.005308,0.014529,0.020104,0.036608,-0.042773,0.005622,0.045505,-0.010229,0.001458,-0.069961,-0.050639,0.043332,-0.050746,-0.083885,0.018574,-0.027424,0.103646,-0.003071,-0.005237,0.045819,-0.007562,-0.057442,0.015487,0.001866,0.043920,0.017178,0.046638,-0.004360,-0.063209,0.007861,-0.065779,-0.026756,0.025221,0.044173,-0.005754,...,0.048164,-0.040121,0.021846,0.004769,-0.036285,-0.058329,0.001414,0.082005,-0.000766,0.102919,-0.024877,0.003432,0.007149,0.025854,0.027951,-0.097248,0.000298,0.023826,0.004917,0.025785,-0.042976,-0.003996,-0.013786,0.035191,-0.010489,-0.006627,-0.090794,-0.073268,-0.087622,-0.062105,-0.083430,-0.010339,0.067203,-0.052454,0.019776,-0.033407,-0.011007,0.072055,0.001881,0.010075
3,0.097561,-0.000109,0.000213,0.063176,-0.057804,0.022388,0.010007,-0.010963,-0.001497,-0.050864,0.040377,0.057871,-0.059007,0.019993,-0.054038,-0.064097,0.034695,-0.058571,-0.073972,0.014602,-0.029369,0.088320,0.073536,0.001778,0.016709,-0.040224,-0.061981,0.015390,0.013797,0.007169,0.016525,0.022672,0.010047,-0.063398,-0.020424,-0.075070,-0.057019,0.039458,0.048060,-0.017432,...,0.058044,-0.009911,0.022132,0.015215,-0.060926,-0.068752,-0.004318,0.070004,0.000758,0.083078,-0.069775,0.006309,-0.011590,0.028734,0.013294,-0.083825,0.044389,0.022830,0.056599,0.003340,-0.054178,-0.039055,0.017346,-0.009694,0.006369,0.026033,-0.077443,-0.065648,-0.058108,-0.023133,-0.082417,-0.014277,0.077998,-0.059772,0.037547,0.010761,0.056996,0.051580,-0.037897,-0.034078
4,0.111851,-0.010639,-0.046339,0.011104,-0.073496,0.009788,0.025644,0.045766,0.048126,-0.043539,-0.009803,0.059885,-0.059836,-0.005724,-0.068129,-0.054173,0.038008,-0.036691,-0.096663,0.014099,-0.021192,0.103184,0.028845,-0.000189,0.033472,-0.007168,-0.059707,0.005258,-0.017568,0.007101,-0.004466,0.036735,-0.007338,-0.045794,0.007524,-0.042120,-0.022910,0.022810,0.063355,-0.019481,...,0.044028,-0.032790,0.043219,-0.001624,-0.042037,-0.052744,-0.031460,0.080702,0.028147,0.105534,-0.043647,0.001934,0.006691,0.033956,0.035109,-0.095423,0.005830,0.025067,0.010496,0.030909,-0.044556,-0.038586,-0.013402,0.012418,0.011574,0.004370,-0.092701,-0.077090,-0.077219,-0.

In [ ]:
embeddings_df.to_csv("embeddings_14000.csv", index=False)
embeddings_df.to_csv("embeddings_index_14000.csv", index=True)

In [ ]:
embeddings_bert = pd.read_csv("embeddings_bert.csv")
embeddings_bert

###### Load embeddings and concatenate to nrc lexicon

In [ ]:
%cd 
%cd /content/drive/MyDrive/emotion_flow_thesis/nrclexicon/
import pandas as pd
embeddings = pd.read_csv("embeddings.csv")
embeddings

In [ ]:
# pd.concat([df1, df4], axis=1).reindex(df1.index)
emolex_df_with_embeddings = pd.concat([emolex_df_short, embeddings], axis=1) 
emolex_df_with_embeddings

###### Take all words with high cosing similarity for a word

In [ ]:
list_of_string_integers_0_511 = []
for i in range(512):
  list_of_string_integers_0_511.append(str(i))


In [ ]:
test_word = "dementieren"
embedding_test_word = np.array(model_1(test_word))
embedding_test_word = list(embedding_test_word[0])
count = 0
for index, row in emolex_df_with_embeddings.iterrows():
    word_in_lexicon = row["word"]
    word_embedding = row[list_of_string_integers_0_511]
    word_embedding = list(word_embedding)
    
    cos_sim = 1 - spatial.distance.cosine(embedding_test_word, word_embedding)
    if cos_sim > 0.80:
      display(count)
      display(word_in_lexicon)
      display(cos_sim)
      count += 1
      



0

'thundering'

0.8078270340913486

1

'ranting'

...

'dealt'

0.8136452984951905

22

'denying'

0.8402614634532988

In [ ]:
%ls


'CHeeSE article example.gdoc'   embeddings.ipynb
 data/                          model.ipynb
 embeddings_bert.csv            NRCLexicon.ipynb
 embeddings_bert_index.csv      nrc_lexicon_with_embeddings.csv
 embeddings.csv                 topics.ipynb
 embeddings_index.csv


In [ ]:
# test text
text = "In der Champions-League-Gruppe mit den Young Boys gewinnt Manchester United auch sein zweites Heimspiel nach einem Rückstand. Gegen Atalanta Bergamo liegen die Red Devils sogar 0:2 zurück, ehe ihnen in der zweiten Halbzeit die Wende gelingt. Der Sieg der Mannschaft von Trainer Ole Gunnar Solskjaer war hochverdient. Standesgemäss war es Cristiano Ronaldo, der mit dem späten 3:2 den Sieg sicherstellte. Der Ausnahmefussballer traf in der 81. mit einem Kopfball aus kurzer Distanz zum Schlussresultat. Bis dahin hatten die Engländer die weitaus besseren und auch mehr Chancen als Atalanta. So trafen Marcus Rashford und Scott McTominay je einmal Latte und Pfosten. RB Salzburg kommt der K.o.-Phase mit langen Schritten näher. Mit einem 3:1-Sieg daheim gegen Wolfsburg holen die Österreicher im dritten Gruppenspiel die Punkte 5 bis 7 – auch dank dem Schweizer Doppeltorschützen Noah Okafor."

In [ ]:
text = "Streit um Spionageverdacht: Russland schliesst seine Nato-Vertretung. Weil sie auch für den Geheimdienst gearbeitet haben sollen, entzog die Nato kürzlich mehreren russischen Diplomaten ihre Akkreditierungen. Moskau zeigte sich verärgert – und reagiert nun mit deutlichen Schritten.  Im Streit um entzogene Akkreditierungen für russische Diplomaten stellt Moskau ab Anfang November die Arbeit seiner ständigen Vertretung bei der Nato in Brüssel ein. Das teilte das russische Aussenministerium am Montag mit."

In [ ]:
text = "Charisma"
emolex_df_with_embeddings = pd.read_csv("embeddings.csv")
emolex_df_with_embeddings

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,...,472,473,474,475,476,477,478,479,480,481,482,483,484,485,486,487,488,489,490,491,492,493,494,495,496,497,498,499,500,501,502,503,504,505,506,507,508,509,510,511
0,0.109317,0.001561,-0.038709,0.011034,-0.057947,0.014078,-0.012465,0.061607,0.033901,-0.019850,-0.036567,0.007742,-0.013863,-0.074603,-0.084126,0.029814,0.004593,-0.056563,-0.092493,0.094409,-0.064294,0.104698,0.062732,0.027869,0.011879,-0.058267,0.001371,-0.056777,-0.020711,0.044578,0.062329,0.054710,-0.057600,-0.061912,0.036521,-0.069653,0.030525,-0.072619,-0.033593,0.021273,...,0.030980,-0.049465,0.029978,0.015868,-0.082230,0.004307,0.018933,0.042480,-0.033291,0.099879,-0.010267,0.036356,-0.049620,-0.026054,0.000915,-0.101772,0.023048,-0.021653,-0.074842,-0.020863,0.054062,-0.004349,-0.073077,0.009696,-0.055970,0.011115,-0.063647,-0.029287,-0.074504,-0.031215,-0.080267,0.017713,0.071654,-0.008148,0.006183,-0.061450,-0.037910,0.083619,-0.030489,0.052017
1,0.106047,-0.027544,0.021171,0.069828,-0.071946,0.005761,0.019075,0.019177,-0.000999,-0.039573,0.058005,0.004947,-0.049800,-0.054871,-0.041643,-0.016565,0.043592,-0.060916,-0.101384,0.090972,0.037509,0.089335,0.045062,0.042826,-0.024961,-0.012006,0.002435,-0.029739,-0.010001,-0.033453,-0.071212,-0.029354,0.005036,-0.056868,-0.020266,-0.046007,0.053158,-0.006658,-0.055757,0.015571,...,0.063003,-0.020009,0.023872,0.049035,-0.082314,-0.047051,-0.014142,-0.024790,-0.047707,0.091288,-0.011316,0.081225,-0.088676,0.038802,0.026768,-0.093349,-0.026427,0.008939,-0.021378,-0.066759,0.009874,-0.005226,-0.062437,-0.022486,-0.075161,0.004215,-0.039382,-0.064156,-0.047073,-0.060916,-0.088460,0.008264,0.061389,-0.006760,0.010137,0.026002,0.036293,0.069374,-0.003556,0.071969
2,0.102450,0.026601,-0.028719,-0.021778,-0.069617,0.012869,-0.020064,0.063414,0.005381,-0.024804,-0.073412,0.031091,-0.034009,0.000852,-0.073866,0.029362,0.075778,-0.067048,-0.088231,0.085642,-0.040075,0.099289,0.046189,-0.001169,0.032040,-0.032451,0.030399,-0.076789,0.006726,-0.016104,-0.015963,0.018930,0.048029,-0.070099,-0.014096,-0.043340,-0.037199,-0.043895,0.014140,-0.012683,...,0.019269,0.022487,0.015034,0.013412,-0.079032,-0.014916,0.007744,-0.034612,-0.053224,0.090190,0.017368,-0.049990,-0.025397,0.014546,0.003744,-0.088337,0.030527,0.041337,0.024898,-0.047597,-0.055134,0.035964,-0.060456,0.013504,-0.021532,0.026825,-0.064709,0.015382,-0.057027,-0.069190,-0.066188,0.029481,0.076109,-0.034778,0.007249,-0.037387,-0.042691,0.067079,-0.025844,0.062328
3,0.104426,-0.017742,-0.028025,-0.042702,-0.078278,0.052751,-0.019000,0.074759,0.027515,-0.021070,-0.083187,0.034721,-0.065755,-0.004861,-0.066471,0.025435,0.005448,-0.070091,-0.083709,0.062874,-0.004058,0.097085,0.054683,-0.053105,0.022473,-0.051704,0.027418,-0.062187,0.022346,-0.019958,-0.002521,-0.004798,0.002326,-0.070612,-0.035092,-0.075825,-0.012454,-0.015831,-0.008222,0.025583,...,0.032795,-0.013993,0.019085,0.039477,-0.082349,-0.051319,-0.034895,-0.041719,-0.019067,0.083620,0.014205,-0.035458,-0.038312,0.021330,0.055533,-0.090524,0.018918,0.051037,-0.032673,-0.044715,-0.031623,0.017882,-0.067044,0.016370,-0.025966,0.017095,-0.046207,0.034618,-0.032920,-0.059186,-0.059598,-0.006696,0.073772,-0.020915,-0.020428,-0.003048,0.003616,0.072193,-0.032594,0.059819
4,0.100910,-0.023130,0.009070,-0.043831,-0.043406,0.013430,-0.016803,0.083027,0.029479,0.008936,-0.007496,-0.004578,-0.026357,-0.012267,-0.047041,-0.033163,0.040400,-0.074469,-0.096610,0.081777,-0.058020,0.089824,0.046935,0.019604,-0.013933,-0.080659,-0.017539,0.021871,0.008629,0.047279,-0.007867,0.054251,-0.014390,-0.065592,-0.064854,-0.086743,0.008201,0.016918,-0.029078,0.014112,...,0.025922,0.060546,0.020595,0.036809,-0.038231,0.052450,0.055897,-0.094344,-0.060118,0.089791,0.000962,0.062148,-0.095022,0.046109,-0.013158,-0.092419,-0.022002,0.002152,-0.016441,0.000041,-0.055089,0.044480,-0.047560,0.008345,-0.057515,-0.006282,-0.079333,0.

In [ ]:
test = 'für'
len(test)

3

In [ ]:
text = "Ausbildung"

In [ ]:

text_splited = text.split()
emotion_list = pd.DataFrame(columns = ['emotion',	'word', 'German-de', 'anger', 'anticipation', 'disgust', 'fear', 'joy', 'sadness', 'surprise', 'trust'])

for word in text_splited:
  if len(word) < 4:
    continue
  display(word)
  embedding_word = np.array(model_1(word))
  embedding_word = list(embedding_word[0])
  count = 0
  words_list = []
  cos_sim_list = []
  for index, row in emolex_df_with_embeddings.iterrows():
      word_in_lexicon = row["word"]
      word_embedding = row[list_of_string_integers_0_511]
      word_embedding = list(word_embedding)

      cos_sim = 1 - spatial.distance.cosine(embedding_word, word_embedding)
      
      if cos_sim > 0.70:
        words_list.append(word_in_lexicon)
        cos_sim_list.append(cos_sim)
        count += 1
  if len(words_list) > 0:
    display(word)
    display(words_list)
    display(cos_sim_list)
      


    


'Ausbildung'

In [ ]:

text_splited = text.split()
emotion_list = pd.DataFrame(columns = ['emotion',	'word', 'German-de', 'anger', 'anticipation', 'disgust', 'fear', 'joy', 'sadness', 'surprise', 'trust'])
#emotion_list.columns = ['anger', 'anticipation', 'disgust', 'fear', 'joy', 'sadness', 'surprise', 'trust']
words_found_counter = 0
words_in_total = 0
for word in text_splited: 
  words_in_total += 1
  if word in emolex_df['German-de'].values:
    word_found = emolex_df.loc[emolex_df['German-de'] == word]
    word_info = word_found.pivot(index=['word' ,'German-de'], columns='emotion', values='emotion-intensity-score').reset_index()
    emotion_list = emotion_list.append(word_info)
    words_found_counter +=1 
    
print("words found: ", words_found_counter)
print("words total: ", words_in_total)
display(emotion_list)
display(emotion_list[['anger', 'anticipation', 'disgust', 'fear', 'joy', 'sadness', 'surprise', 'trust']].mean())

words found:  6
words total:  68


,emotion,word,German-de,anger,anticipation,disgust,fear,joy,sadness,surprise,trust
0,NaN,argument,Streit,0.545,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,dispute,Streit,0.531,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,quarrel,Streit,0.781,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,strife,Streit,0.625,NaN,NaN,NaN,NaN,NaN,NaN,NaN
0,NaN,labored,gearbeitet,NaN,NaN,NaN,NaN,NaN,0.25,NaN,NaN
0,NaN,angered,verärgert,0.916,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,annoyed,verärgert,0.485,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,disgruntled,verärgert,0.693,NaN,0.406,NaN,NaN,0.712,NaN,NaN
3,NaN,peeved,verärgert,0.688,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,pissoff,verärgert,0.875,NaN,NaN,NaN,NaN,NaN,NaN,NaN


anger           0.647143
anticipation    0.566500
disgust         0.406000
fear                 NaN
joy             0.106000
sadness         0.508000
surprise        0.312500
trust           0.414000
dtype: float64

In [ ]:
 text = "Konflikt"

In [ ]:
text_splited = text.split()
emotion_list = pd.DataFrame(columns = ['emotion',	'word', 'German-de', 'anger', 'anticipation', 'disgust', 'fear', 'joy', 'sadness', 'surprise', 'trust'])
#emotion_list.columns = ['anger', 'anticipation', 'disgust', 'fear', 'joy', 'sadness', 'surprise', 'trust']
words_found_counter = 0
words_in_total = 0
for word in text_splited: 
  words_in_total += 1
  if word in emolex_df['German-de'].values:
    word_found = emolex_df.loc[emolex_df['German-de'] == word]
    word_info = word_found.pivot(index=['word' ,'German-de'], columns='emotion', values='emotion-intensity-score').reset_index()
    emotion_list = emotion_list.append(word_info)
    words_found_counter +=1 
    
print("words found: ", words_found_counter)
print("words total: ", words_in_total)
display(emotion_list)


words found:  1
words total:  1


,emotion,word,German-de,anger,anticipation,disgust,fear,joy,sadness,surprise,trust
0,NaN,conflict,Konflikt,0.578,NaN,NaN,0.641,NaN,0.469,NaN,NaN
